In [1]:
!pip install geemap

  Obtaining dependency information for geemap from https://files.pythonhosted.org/packages/a0/81/3b924526fd7bb797e2b619e1adab16260e5cbb2727ab5a6b09dd3acf31d0/geemap-0.32.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for bqplot from https://files.pythonhosted.org/packages/ad/f5/717a355a47e8ba28ff9fcbe63d3014b55a8d20f3983582205085e85e2869/bqplot-0.12.43-py2.py3-none-any.whl.metadata
  Obtaining dependency information for colour from https://files.pythonhosted.org/packages/74/46/e81907704ab203206769dee1385dc77e1407576ff8f50a0681d0a6b541be/colour-0.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for earthengine-api>=0.1.347 from https://files.pythonhosted.org/packages/30/17/d9679d5d8fec7763c1725afcedbdf4fb62267b3c5210eedadb0b99c0c49e/earthengine_api-0.1.405-py3-none-any.whl.metadata
  Obtaining dependency information for eerepr>=0.0.4 from https://files.pythonhosted.org/packages/0b/71/5a5e36ad755e3e3081b43acf9ac2a50d1f6f88211aa278bb80114d0b6cd6/eer


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [3]:
import ee
import geemap
import pandas as pd

# Inicializar Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


Successfully saved authorization token.


In [9]:
# Definir la colección de imágenes y los años de interés
colID = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG"
start = 2012
end = 2022

years = ee.List.sequence(start, end)

In [5]:
# Función para reducir las imágenes anuales a una mediana
def viirs_annual_median_reduce(year):
    return ee.ImageCollection(colID).filter(
        ee.Filter.calendarRange(year, year, "year")).select("avg_rad").median().set('year', year)

In [10]:
# Mapear la función sobre la lista de años
yearComps = ee.ImageCollection.fromImages(years.map(viirs_annual_median_reduce))

# Inicializar un diccionario para almacenar los resultados
data = {'Year': [], 'Average Radiation': []}

In [11]:
# Extraer los valores promedio de para cada año
for year in range(start, end + 1):
    img = yearComps.filterMetadata("year", "equals", year).first()
    avg_rad = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.Point(-70.51276,-17.42283).buffer(20000),
        scale=1000
    ).getInfo().get('avg_rad')
    
    data['Year'].append(year)
    data['Average Radiation'].append(avg_rad)

In [12]:
df = pd.DataFrame(data)
df.to_excel('luminosidad_230302.xlsx', index=False)